In [1]:
!pip install --upgrade accelerate transformers trl evaluate rouge_score peft
!pip install datasets==2.21.0
!pip install pyyaml wandb
!pip freeze > requirements.txt

from google.colab import drive
drive.mount('/content/drive')

!pip install uv

!uv init --name kyivstar-llm-summarization --no-readme

!uv add torch transformers datasets accelerate peft trl numpy pandas pyyaml tqdm wandb evaluate rouge-score

!uv lock

from google.colab import files
files.download('pyproject.toml')
files.download('uv.lock')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 52.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.21.0
    Uninstalling datasets-2.21.0:
      Successfully uninstalled datasets-2.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
Using

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/kyivstar-test-task.zip'

extract_path = '/content/project'

os.makedirs(extract_path, exist_ok=True)

print(f"Распаковываем архив из {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Архив распакован в {extract_path}")

print("\nСодержимое /content/project:")
!ls -la /content/project

print("\nДерево директорий:")
!find /content/project -maxdepth 2 -type d

print("\nИщем config.yaml:")
!find /content/project -name "config.yaml"

print("\nИщем Python файлы проекта:")
!find /content/project -name "entry_point.py"
!find /content/project -name "data_preparation_and_generation.py"

Распаковываем архив из /content/drive/MyDrive/kyivstar-test-task.zip...
Архив распакован в /content/project

Содержимое /content/project:
total 12
drwxr-xr-x  3 root root 4096 Nov 12 17:05 .
drwxr-xr-x  1 root root 4096 Nov 12 17:05 ..
drwxr-xr-x 13 root root 4096 Nov 12 17:08 kyivstar-test-task

Дерево директорий:
/content/project
/content/project/kyivstar-test-task
/content/project/kyivstar-test-task/outputs
/content/project/kyivstar-test-task/checkpoints
/content/project/kyivstar-test-task/sft_output
/content/project/kyivstar-test-task/media
/content/project/kyivstar-test-task/models
/content/project/kyivstar-test-task/llm_summarize
/content/project/kyivstar-test-task/notebooks
/content/project/kyivstar-test-task/scripts
/content/project/kyivstar-test-task/data
/content/project/kyivstar-test-task/config
/content/project/kyivstar-test-task/grpo_output

Ищем config.yaml:
/content/project/kyivstar-test-task/config/config.yaml

Ищем Python файлы проекта:
/content/project/kyivstar-test-t

In [3]:
from google.colab import userdata
import os
import wandb
from huggingface_hub import login

hf_token = userdata.get('HF_ACSESS_TOKEN')
wandb_key = userdata.get('WANDB_API_KEY')

os.environ['HF_TOKEN'] = hf_token
os.environ['WANDB_API_KEY'] = wandb_key

login(token=hf_token)
wandb.login(key=wandb_key)



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dmytro-stepanchuk-cs (dmytro-stepanchuk-cs-igor-sikorsky-kyiv-polytechnic-inst) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
import yaml

with open('/content/project/kyivstar-test-task/config/config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("Конфигурация загружена:")
print(f"  - Базовая модель: {config['models']['base_model']['name']}")
print(f"  - Train samples: {config['dataset']['subset_sizes']['train']}")
print(f"  - SFT epochs: {config['sft']['training']['num_train_epochs']}")
print(f"  - GRPO epochs: {config['grpo']['training']['num_train_epochs']}")

Конфигурация загружена:
  - Базовая модель: google/gemma-3-1b-it
  - Train samples: 10000
  - SFT epochs: 1
  - GRPO epochs: 1


In [ ]:
import subprocess
import sys

script_path = '/content/project/kyivstar-test-task/scripts/entry_point.py'

process = subprocess.Popen(
    [sys.executable, script_path, '--stages','data', 'sft'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    cwd='/content/project/kyivstar-test-task'
)

for line in process.stdout:
    print(line, end='')

return_code = process.wait()

print("\n" + "="*80)
if return_code != 0:
    print(f"❌ Ошибка! Return code: {return_code}")

In [ ]:
import subprocess
import sys

script_path = '/content/project/kyivstar-test-task/scripts/entry_point.py'



process = subprocess.Popen(
    [sys.executable, script_path, '--full', '--stages', 'grpo'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    cwd='/content/project/kyivstar-test-task'
)

for line in process.stdout:
    print(line, end='')

return_code = process.wait()

print("\n" + "="*80)
if return_code != 0:
    print(f"❌ Ошибка! Return code: {return_code}")

2025-11-12 09:25:55.680244: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 09:25:55.700602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762939555.722572   23394 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762939555.729169   23394 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762939555.746380   23394 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
import subprocess
import sys

script_path = '/content/project/kyivstar-test-task/scripts/entry_point.py'



process = subprocess.Popen(
    [sys.executable, script_path, '--stages','eval'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    cwd='/content/project/kyivstar-test-task'
)

for line in process.stdout:
    print(line, end='')

return_code = process.wait()

print("\n" + "="*80)
if return_code != 0:
    print(f"❌ Ошибка! Return code: {return_code}")

2025-11-12 17:09:28.142670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 17:09:28.160720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762967368.181976    2400 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762967368.188444    2400 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762967368.205118    2400 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
import shutil
import os

project_dir = '/content/project/kyivstar-test-task'

output_path = '/content/drive/MyDrive/kyivstar-test-task_COMPLETED'

print(f"Архивируем директорию: {project_dir}")
print(f"Целевой архив: {output_path}.zip")

shutil.make_archive(
    base_name=output_path,
    format='zip',
    root_dir=os.path.dirname(project_dir),
    base_dir=os.path.basename(project_dir)
)

print(f"\n✓ Архив создан успешно!")
print(f"Размер архива:")

archive_path = output_path + '.zip'
if os.path.exists(archive_path):
    size_mb = os.path.getsize(archive_path) / (1024 * 1024)
    print(f"  {archive_path}")
    print(f"  Размер: {size_mb:.2f} MB")
else:
    print(f"  Ошибка: архив не найден!")

print("\nСодержимое архива (первые 20 файлов):")
import zipfile
with zipfile.ZipFile(archive_path, 'r') as zip_ref:
    files = zip_ref.namelist()[:20]
    for f in files:
        print(f"  - {f}")
    if len(zip_ref.namelist()) > 20:
        print(f"  ... и ещё {len(zip_ref.namelist()) - 20} файлов")

Архивируем директорию: /content/project/kyivstar-test-task
Целевой архив: /content/drive/MyDrive/kyivstar-test-task_COMPLETED.zip

✓ Архив создан успешно!
Размер архива:
  /content/drive/MyDrive/kyivstar-test-task_COMPLETED.zip
  Размер: 812.77 MB

Содержимое архива (первые 20 файлов):
  - kyivstar-test-task/
  - kyivstar-test-task/checkpoints/
  - kyivstar-test-task/config/
  - kyivstar-test-task/data/
  - kyivstar-test-task/grpo_output/
  - kyivstar-test-task/llm_summarize/
  - kyivstar-test-task/media/
  - kyivstar-test-task/models/
  - kyivstar-test-task/notebooks/
  - kyivstar-test-task/outputs/
  - kyivstar-test-task/scripts/
  - kyivstar-test-task/sft_output/
  - kyivstar-test-task/pyproject.toml
  - kyivstar-test-task/entry_point_as_google_colab.ipynb
  - kyivstar-test-task/README.md
  - kyivstar-test-task/pipeline.log
  - kyivstar-test-task/uv.lock
  - kyivstar-test-task/outputs/.ipynb_checkpoints/
  - kyivstar-test-task/outputs/grpo_predictions.jsonl
  - kyivstar-test-task/ou